## Planning
*This notebook is for an intial exploration of the data only - processing and full analysis is done in main*
# Key Objectives:
1. Clean the data
2. Complete features selection
3. Generate exploratory plots using matplotlib
4. Generate interactive plots using plotly

### Imports and Setup

In [9]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List, Dict

plt.style.use("ggplot")

CWD = Path.cwd()
DATA_DIR: Path = CWD.parent.parent / "data"
SURVEY_PATH: Path = DATA_DIR / "survey" /"survey_results_public_2024.csv"
SURVEY_SCHEMA: Path = DATA_DIR / "survey" / "survey_results_schema_dropcols.csv"

In [10]:
survey_raw: pd.DataFrame = pd.read_csv(SURVEY_PATH)
survey_schema: pd.DataFrame = pd.read_csv(SURVEY_SCHEMA)

### Preprocessing

In [8]:
def get_column_variants(list1: list, list2: list) -> list:
    """Get variant names of columns in the list2 which are contained in list1 with additional modifications e.g. suffixes"""
    return [x for x in list1 if any(y in x and x!=y for y in list2)]


def generate_drop_colnames(stem: str) -> List[str]:
    """Create additional variants of columns by adding suffixes"""
    return stem + "WantToWorkWith", stem + "Admired"


def drop_unwanted_cols(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drop columns using survey schema "drop" column (created manually).
    Additional manual correction was required because some columns didn't map
    / new columns were added compared to the reported schema.
    """
    dropcols = (
        survey_schema
        .loc[survey_schema["drop"] == 1, "qname"]
        .to_list()
    )
    df_columns = df.columns.to_list()  # survey columns don't match schema
    unmatched_colums = list(set(dropcols) - set(df_columns))
    matched_dropcols = [elem for elem in dropcols if elem not in unmatched_colums]
    cols_to_drop_new = get_column_variants(list1=df_columns, list2=dropcols)
    dropcols = matched_dropcols + cols_to_drop_new
    missed_cols = [
        "Language", "Database", "Platform",
        "Webframe", "Embedded", "MiscTech",
        "ToolsTech", "NEWCollabTools", "OfficeStackAsync"
    ]
    dropcols += [x for stem in missed_cols for x in generate_drop_colnames(stem)]
    result = df.drop(columns=dropcols, axis=1, inplace=False)
    return result

In [4]:
# Apply schema to drop columns
df = drop_unwanted_cols(survey_raw) 

In [5]:
# Clean and filter the data
# Drop rows where any of the following columns are null
df = drop_unwanted_cols(survey_raw)
df = df.dropna(subset=["ConvertedCompYearly"])
filter_values: dict = {
    "Check": ["Apples"],
    "MainBranch": ["I am a developer by profession"],
    "Employment": ["Employed, full-time"],
    "DevType": [
        "Data or business analyst",
        "Data scientist or machine learning specialist",
        "Data or business analyst",
        "Developer, AI"
        ]
}
for k,v in filter_values.items():
    df = df[df[k].isin(v)]
df = df.drop([
    "Age", "Employment", "OrgSize", "RemoteWork", "Check", "EdLevel",
    "YearsCode", "Country", "Currency", "CompTotal", "OpSysPersonal use",
    "ICorPM", "WorkExp", "Industry", "YearsCodePro", "MainBranch", 
    "ResponseId", "DevType"
], axis=1)

print(df.columns)

Index(['LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith',
       'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith',
       'EmbeddedHaveWorkedWith', 'MiscTechHaveWorkedWith',
       'ToolsTechHaveWorkedWith', 'NEWCollabToolsHaveWorkedWith',
       'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith',
       'ConvertedCompYearly', 'JobSat'],
      dtype='object')


In [25]:
# Dump column names to pickle
import pickle
with open("columnnames.pickle", "wb") as f:
    pickle.dump(list(df.columns), f)

### Extract Features and Split Dataset

In [26]:
split_columns = []
for col in df.columns[:-2]:
    split_columns.append(df[col].str.get_dummies(sep=';').add_prefix(f"{col}_"))
split_df = pd.concat(split_columns, axis=1)
display(split_df)

,LanguageHaveWorkedWith_Ada,LanguageHaveWorkedWith_Apex,LanguageHaveWorkedWith_Assembly,LanguageHaveWorkedWith_Bash/Shell (all shells),LanguageHaveWorkedWith_C,LanguageHaveWorkedWith_C#,LanguageHaveWorkedWith_C++,LanguageHaveWorkedWith_Clojure,LanguageHaveWorkedWith_Cobol,LanguageHaveWorkedWith_Dart,...,OfficeStackAsyncHaveWorkedWith_Monday.com,OfficeStackAsyncHaveWorkedWith_Notion,OfficeStackAsyncHaveWorkedWith_Obsidian,OfficeStackAsyncHaveWorkedWith_Redmine,OfficeStackAsyncHaveWorkedWith_Shortcut,OfficeStackAsyncHaveWorkedWith_Smartsheet,OfficeStackAsyncHaveWorkedWith_Stack Overflow for Teams,OfficeStackAsyncHaveWorkedWith_Trello,OfficeStackAsyncHaveWorkedWith_Wikis,OfficeStackAsyncHaveWorkedWith_YouTrack
379,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
585,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
598,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
746,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
785,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40872,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40944,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
40975,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [7]:
from sklearn.model_selection import train_test_split
X = split_df
y = df[["ConvertedCompYearly", "JobSat"]]  # Target variable(s)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y1_train = y_train["ConvertedCompYearly"]
y2_train = y_train["JobSat"]
y1_test = y_test["ConvertedCompYearly"]
y2_test = y_test["JobSat"]
cat_attribs = list(X)

### Model Training

#### Linear Regression

In [8]:
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

param_grid_lr = [
    {
        'fit_intercept': [True, False],
        'positive': [True],  # Constrain coefficients to be positive
    }
]
grid_search_lr = GridSearchCV(LinearRegression(), param_grid=param_grid_lr, cv=5, 
                           scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_lr.fit(X_train, y1_train)

param_grid_ridge = [
    {
        'fit_intercept': [True, False],
        'positive': [True],
        'alpha': [0.1, 1, 10, 100]  # Regularization strength (only for Ridge)
    }
]
grid_search_ridge = GridSearchCV(Ridge(), param_grid=param_grid_ridge, cv=5, 
                           scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_ridge.fit(X_train, y1_train)

param_grid_elastic = [
    {
        'alpha': [0.01, 0.1, 1],  # Regularization strength
        'l1_ratio': [0.3, 0.5, 0.7],  # L1/L2 mix (0=L2, 1=L1)
        'positive': [True],
        'max_iter': [1000, 5000, 10000]  # Ensure convergence for large datasets
    }
]
# Commenting out elastic net as not sure it makes sense to eliminate skills
# grid_search_elastic = GridSearchCV(ElasticNet(), param_grid=param_grid_elastic, cv=5, 
#                            scoring='neg_mean_squared_error', n_jobs=-1)
# grid_search_elastic.fit(X_train, y1_train)


best_model = max([
    grid_search_lr, grid_search_ridge, #grid_search_elastic
    ], key=lambda gs: gs.best_score_)
lreg = best_model.best_estimator_

#### Random Forest

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30, 100], 'max_features': [100, 200, 312]},
    {'bootstrap': [False], 'n_estimators': [3, 10, 30, 100], 'max_features': [100, 200, 312]},
]

rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, 
                           scoring='neg_mean_squared_error', n_jobs=-1, )
grid_search.fit(X_train, y1_train)
rf = grid_search.best_estimator_

### XGBoost

In [ ]:
from xgboost import XBGRegressor

xgb_r = XGBRegressor()

#### Model Evaluation

In [10]:
from sklearn.model_selection import cross_val_score

def display_scores(scores: np.array):
    return f"Scores: {scores}\nMean: {scores.mean()}\nStandard deviation: {scores.std()}"


# Evaluate linear regression performance
# Data engineering only, mean:98670, std: 45171
lin_reg_rmse_scores = np.sqrt(-cross_val_score(lreg, X_train, y1_train,
                        scoring="neg_mean_squared_error", cv=10))
print(f"Linear Regression:\n{display_scores(lin_reg_rmse_scores)}")

# Evaluate random forest performance
# Data engineering only, mean: 106085, std: 41032
forest_reg_rmse_scores = np.sqrt(-cross_val_score(rf, X_train, y1_train,
                            scoring="neg_mean_squared_error", cv=10))
print(f"Random Forest:\n{display_scores(forest_reg_rmse_scores)}")

Linear Regression:
Scores: [203991.39138031  96603.38923712 119497.18705362  61049.83335781
  49488.63547606  63530.82389754 128008.89814894  61851.38729584
 129400.36226336  73283.79073386]
Mean: 98670.5698844451
Standard deviation: 45171.53894126881
Random Forest:
Scores: [202451.4895911  120658.75628646 120890.09030826  75344.25059005
  57097.54396499  70793.9532489  125586.32590361  72674.03228477
 126541.98278475  78607.05052756]
Mean: 105064.54754904534
Standard deviation: 41192.25046872937


In [11]:
linear_reg_result = pd.DataFrame(list(zip(split_df.columns, lreg.coef_)), columns=["skill", "coeff_linear_regression"])
linear_reg_result = linear_reg_result.sort_values(by="coeff_linear_regression", ascending=False)
linear_reg_result.to_csv("linRegCoeffs.csv")

forest_reg_result = pd.DataFrame(list(zip(split_df.columns, rf.feature_importances_)), columns=["skill", "coeff_random_forest"])
forest_reg_result = forest_reg_result.sort_values(by="coeff_random_forest", ascending=False)
forest_reg_result.to_csv("forestRegCoeffs.csv")

combined_reg_results = pd.merge(linear_reg_result, forest_reg_result, on="skill", how="outer")
combined_reg_results["coeff_product"] = combined_reg_results["coeff_linear_regression"] * combined_reg_results["coeff_random_forest"]
combined_reg_results = combined_reg_results.sort_values(by="coeff_product", ascending=False)
combined_reg_results.to_csv("combinedRegCoeffs.csv")
display(combined_reg_results)

,skill,coeff_linear_regression,coeff_random_forest,coeff_product
141,NEWCollabToolsHaveWorkedWith_DataGrip,8841.024302,0.043233,382.225570
93,LanguageHaveWorkedWith_Solidity,6135.196663,0.030744,188.621333
245,ToolsTechHaveWorkedWith_Ant,8102.857769,0.021020,170.321903
258,ToolsTechHaveWorkedWith_Make,8510.732122,0.019484,165.823145
162,NEWCollabToolsHaveWorkedWith_RubyMine,5559.337123,0.027930,155.270136
...,...,...,...,...
14,DatabaseHaveWorkedWith_Firebird,0.000000,0.000017,0.000000
12,DatabaseHaveWorkedWith_Elasticsearch,0.000000,0.005113,0.000000
11,DatabaseHaveWorkedWith_Dynamodb,0.000000,0.004128,0.000000
303,WebframeHaveWorkedWith_Solid.js,0.000000,0.000222,0.000000


In [12]:
# TODO:
# Refactor this code into a pipeline which can handle more years of data
# Get more years of data from the survey
# Retrain models with more data
# Look at trends over time
# Look at shapely additive explanations and consider mixed effects models
# Begin exploring linkedin ads to compare demand vs supply
# Write all of this up into a dashboard/ blog post